In [2]:
from threaded_sweep import Sweep1D, Sweep0D, Sweep2D
from daq_driver import Daq, DaqAOChannel, DaqAIChannel
import nidaqmx
import numpy as np
import qcodes as qc
import time
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.database import initialise_or_create_database_at
from qcodes.dataset.data_export import get_data_by_id

In [3]:
# initialize the DAQ
daq = Daq("Dev1", "testdaq")

In [4]:
# Initialize the database you want to save data to
try:
    experimentName = 'test_thread_sweep2d'
    sampleName = '0'
    initialise_or_create_database_at('C:\\Users\\Nanouser\\Documents\\MeasureIt\\Databases\\Carbon_resistor_wo_filter.db')
    qc.new_experiment(name=experimentName, sample_name=sampleName)
except:
    print("Error opening database")
    daq.device.reset_device()
    daq.__del__()
    quit()

In [5]:
daq.ai0.voltage.label="I"
daq.ai1.voltage.label="Vxx"

%matplotlib qt

In [ ]:
def track_setpoint(sweep, lines, setpoint, steps = 100):
    if isinstance(lines, list):
        fwd_line = lines[0]
        bkw_line = lines[1]
    else:
        fwd_line = lines
        
    def find_point(line):
        min_diff = float("inf")
        best_x = 0
        best_y = 0
        x_data, y_data = line.get_data()
        
        for i,y in enumerate(y_data):
            if abs(y-setpoint)<min_diff:
                min_diff = abs(y-setpoint)
                best_x = x_data[i]
                best_y = y
        return (best_x, best_y)
    
    fwd_peak = find_point(fwd_line)
    bkw_peak = find_point(bkw_line)
    
    avg_peak_pos = (fwd_peak[0]+bkw_peak[0])/2
    
    sweep.begin = avg_peak_pos - (steps/2*sweep.step)
    sweep.end = sweep.begin + sweep.step*steps 

In [ ]:
def track_peak(sweep, lines, steps = 100):
    fwd_line = lines[0]
    bkw_line = lines[1]
    
    def get_peak(line):
        max_y = float("-inf")
        max_x = 0
        x_data, y_data = line.get_data()
        
        for i,y in enumerate(y_data):
            if y > max_y:
                max_y = y
                max_x = x_data[i]
        return (max_x, max_y)
            
    fwd_peak = get_peak(fwd_line)
    bkw_peak = get_peak(bkw_line)
    
    avg_peak_pos = (fwd_peak[0]+bkw_peak[0])/2
    
    sweep.begin = avg_peak_pos - (steps/2*sweep.step)
    sweep.end = sweep.begin + sweep.step*steps 

In [ ]:
import numpy as np
import matplotlib as mpl

def track_jump(sweep, lines, pos_jump=True, steps=100):
    fwd_line = lines[0]
    bkw_line = lines[1]
    
    d_fwd_line = mpl.lines.Line2D(fwd_line.get_xdata(), np.gradient(fwd_line.get_ydata()))
    d_bkw_line = mpl.lines.Line2D(bkw_line.get_xdata(), np.gradient(bkw_line.get_ydata()))
    
    def get_peak(line):
        if pos_jump:
            best_y = float("-inf")
        else:
            best_y = float("inf")
        best_x = 0
        x_data, y_data = line.get_data()
        
        for i,y in enumerate(y_data):
            if pos_jump and y > best_y:
                best_y = y
                best_x = x_data[i]
            else if not pos_jump and y < best_y:
                best_y = y
                best_x = x_data[i]
        return (best_x, best_y)
    
    d_fwd_peak = get_peak(d_fwd_line)
    d_bkw_peak = get_peak(d_bkw_line)
    
    avg_peak_pos = (d_fwd_peak[0]+d_bkw_peak[0])/2
    
    sweep.begin = avg_peak_pos - (steps/2)*sweep.step
    sweep.end = sweep.begin + sweep.step*steps 

In [ ]:
# testing 2d sweep
# Set the sweep parameters -
# [ Parameter, start point, end point, step ]
in_params = [daq.ao0.voltage, 0, 1, 0.01]
out_params = [daq.ao1.voltage, 0, 1, 0.1]
s = Sweep2D(in_params, out_params, inter_delay = 0.05, update_func = track_peak)
s.follow_param(daq.ai0.voltage, daq.ai1.voltage, daq.ai2.voltage)
s._create_measurement()

In [ ]:
s.start()